In [ ]:
import fastf1
import fastf1.plotting

import pandas as pd
import numpy as np
import os  
from datetime import timedelta
import csv
from fastf1.ergast import Ergast

ergast = Ergast()

year = 2025
race_number = 1

#year = int(input("Year ? "))
#race_number = int(input("Race Number ? (1-24) "))

session_fp1= fastf1.get_session(year, race_number, 'FP1')
session_fp1.load(weather=True)

session_fp2= fastf1.get_session(year, race_number, 'FP2')
session_fp2.load(weather=True)

session_fp3= fastf1.get_session(year, race_number, 'FP3')
session_fp3.load(weather=True)

#pit = ergast.get_pit_stops(season = year, round = race_number )
teams = fastf1.plotting.list_team_names(session_fp1)
event_name = session_fp1.event.EventName

req         WARNING 	DEFAULT CACHE ENABLED! (1.6 GB) /home/kurios/.cache/fastf1
core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loadin

In [5]:
session_fp1.session_status

,Time,Status
0,0 days 00:00:11.755000,Inactive
1,0 days 00:14:32.179000,Started
2,0 days 00:33:02.752000,Aborted
3,0 days 00:38:33.627000,Inactive
4,0 days 00:39:32.166000,Started
5,0 days 00:51:57.748000,Aborted
6,0 days 01:02:37.768000,Inactive
7,0 days 01:03:32.171000,Started
8,0 days 01:14:32.184000,Finished
9,0 days 01:20:45.416000,Finalised


In [ ]:
practice_track_status = {}

for practice_session in (session_fp1, session_fp2, session_fp3):
    laps = practice_session.laps
    stints = laps[["Driver", "Stint", "Compound", "LapNumber", "LapTime"]]
    stints = stints.groupby(["Driver", "Stint", "Compound"]).agg(
        LapCount=('LapNumber', 'count'),
        FirstLap=('LapNumber', 'min'),
        LastLap=('LapNumber', 'max')
    ).reset_index()

    print(practice_session)
    print('\nAir Temp Range : (' + str(min(practice_session.weather_data['AirTemp'])) + '-' + str(max(practice_session.weather_data['AirTemp'])) +')')
    print('Humidity Range : (' + str(min(practice_session.weather_data['Humidity'])) + '-' + str(max(practice_session.weather_data['Humidity'])) +')')
    print('Track Temp Range : (' + str(min(practice_session.weather_data['TrackTemp'])) + '-' + str(max(practice_session.weather_data['TrackTemp'])) +')\n')
    
    teams = fastf1.plotting.list_team_names(practice_session)
    
    #for driver in practice_session.drivers:
    for idx, team in enumerate(teams):
        team_drivers = fastf1.plotting.get_driver_abbreviations_by_team(team, session=practice_session)

        driver_info_1 = practice_session.get_driver(team_drivers[0])
        driver_info_2 = practice_session.get_driver(team_drivers[1])
        
        for driver_info in (driver_info_1, driver_info_2):
            driver = driver_info['DriverNumber']
            
            epsilon = 1e-6
            practice_session.car_data[driver]['is_driving'] = (practice_session.car_data[driver]['Speed'] > epsilon) & (practice_session.car_data[driver]['nGear'] > 0)
            practice_session.car_data[driver][practice_session.car_data[driver]['is_driving'] == True]
            practice_session.car_data[driver]['block_start'] = practice_session.car_data[driver]['is_driving'] & (~practice_session.car_data[driver]['is_driving'].shift(1, fill_value=False))
            practice_session.car_data[driver]['block_id'] = practice_session.car_data[driver]['block_start'].cumsum()
            df_driving_only = practice_session.car_data[driver][practice_session.car_data[driver]['is_driving']].copy()

            if not df_driving_only.empty:
                fastest_lap = practice_session.laps.pick_drivers(driver).pick_fastest()
                driver = driver_info["Abbreviation"]

                driving_block_durations = df_driving_only[df_driving_only['block_id'] > 0].groupby('block_id')['SessionTime'].apply(lambda x: x.iloc[-1] - x.iloc[0])
                min_block_duration = pd.Timedelta(minutes=1)
                filtered_driving_block_durations = driving_block_durations[driving_block_durations >= min_block_duration]
                total_run_time = str(filtered_driving_block_durations.sum())[10:-3]
                print(f'{driver}:')
                try:
                    print(f'Fastest Lap : ' + str(fastest_lap['LapTime'])[10:-3])
                except:
                    print(f"No fastest lap found.\n")

                driver_stints = stints[stints['Driver'] == driver]
                for index, row in driver_stints.iterrows():
                    driver_lap = practice_session.laps.pick_drivers(driver).pick_laps(range(int(row['FirstLap']), int(row['LastLap'])+1)).pick_quicklaps()
                    try:
                        driver_avg_lap = str(timedelta(seconds = driver_lap['LapTime'].dt.total_seconds().median()))[3:-3]
                        compound = stints['Compound'][index]
                        driver = practice_session.get_driver(driver)["Abbreviation"]
                        print(f'Average Quick Lap on {compound} tyre : ' + str(driver_avg_lap))
                    except:
                        print(f'Average Quick Lap on {compound} tyre : ' + 'No valid lap times')
                if not filtered_driving_block_durations.empty:
                    print(f"Total Run Time: {total_run_time}\n")
                else:
                    print(f"No driving blocks found that are at least 1 minute long.")
                    print(f"Total Run Time: {total_run_time}\n")

            else:
                print(f"No driving data found (Speed > 0 and Gear > 0).")

2025 Season Round 1: Australian Grand Prix - Practice 1

Air Temp Range : (23.8-24.8)
Humidity Range : (44.0-54.0)
Track Temp Range : (39.2-44.6)

VER:
Fastest Lap : 01:17.696
Average Quick Lap on MEDIUM tyre : 1:19.149
Average Quick Lap on SOFT tyre : 1:17.696
Average Quick Lap on SOFT tyre : 2:06.808
Average Quick Lap on MEDIUM tyre : 1:22.953
Total Run Time: 36:19.470

NOR:
Fastest Lap : 01:17.252
Average Quick Lap on MEDIUM tyre : 1:18.765
Average Quick Lap on MEDIUM tyre : 1:20.514
Average Quick Lap on SOFT tyre : 1:17.385
Total Run Time: 34:28.195

BOR:
Fastest Lap : 01:18.438
Average Quick Lap on MEDIUM tyre : 1:23.484
Average Quick Lap on MEDIUM tyre : No valid lap times
Average Quick Lap on MEDIUM tyre : 1:19.676
Average Quick Lap on SOFT tyre : 1:19.914
Total Run Time: 40:00.635

HAD:
Fastest Lap : 01:17.847
Average Quick Lap on MEDIUM tyre : 1:19.510
Average Quick Lap on MEDIUM tyre : 1:18.218
Average Quick Lap on MEDIUM tyre : 1:21.591
Average Quick Lap on MEDIUM tyre : 1:2

In [11]:
for practice_session in (session_fp1, session_fp2, session_fp3):
    #for driver in practice_session.drivers:
        driver = '16'
        fastest_lap = practice_session.laps.pick_drivers(driver).pick_fastest()
        fastest_lap_tel = fastest_lap.get_car_data()
        driver = practice_session.get_driver(driver)["Abbreviation"]
        compound = fastest_lap['Compound']
        tyre_life = fastest_lap['TyreLife']
        full_throttle = round(len(np.where(fastest_lap_tel['Throttle'].values >= 90)[0])/len(fastest_lap_tel)*100)
        brake = round(len(np.where(fastest_lap_tel['Brake'] == True)[0])/len(fastest_lap_tel)*100)
        cornering = 100 - full_throttle - brake
        i1_trap_speed = int(fastest_lap['SpeedI1'])
        i2_trap_speed = int(fastest_lap['SpeedI2'])
        fl_trap_speed = int(fastest_lap['SpeedFL'])
        speed_trap_speed = int(fastest_lap['SpeedST'])
        compound = fastest_lap['Compound']

        print(driver)
        print(str(fastest_lap['LapTime'])[10:-3])
        print('Compound : ' + compound)
        print('Tyre Life : ' + str(tyre_life))
        print('Full Throttle : ' + str(full_throttle))
        print('Brake : ' + str(brake))
        print('Cornering : ' + str(cornering))
        print('I1 Trap Speed : ' + str(i1_trap_speed) + ' kp/h')
        print('I2 Trap Speed : ' + str(i2_trap_speed) + ' kp/h')
        print('FL Trap Speed : ' + str(fl_trap_speed) + ' kp/h')
        print('Speed Trap Speed : ' + str(speed_trap_speed) + ' kp/h')

LEC
01:17.461
Compound : SOFT
Tyre Life : 2.0
Full Throttle : 72
Brake : 15
Cornering : 13
I1 Trap Speed : 286 kp/h
I2 Trap Speed : 314 kp/h
FL Trap Speed : 305 kp/h
Speed Trap Speed : 321 kp/h
LEC
01:16.439
Compound : SOFT
Tyre Life : 5.0
Full Throttle : 72
Brake : 13
Cornering : 15
I1 Trap Speed : 287 kp/h
I2 Trap Speed : 315 kp/h
FL Trap Speed : 305 kp/h
Speed Trap Speed : 322 kp/h
LEC
01:16.188
Compound : SOFT
Tyre Life : 5.0
Full Throttle : 70
Brake : 14
Cornering : 16
I1 Trap Speed : 288 kp/h
I2 Trap Speed : 322 kp/h
FL Trap Speed : 309 kp/h
Speed Trap Speed : 324 kp/h


On average an F1 car consume 2.5L per lap about 1.7kg of fuel per lap,
10 kg will have a 0.15% impact on lap time,
Maximum 110kg fuel 

Engine mode can be estimate based on lap at speed trap 
Can increase top speed by 5–15 km/h, depending on deployment length and car setup
Using ERS can gain you between 0.4% and 0.7% depending on the speed of the circuit

In [6]:
print(fastest_lap_tel)

                       Date      RPM  Speed  nGear  Throttle  Brake  DRS  \
0   2025-03-15 02:16:16.072  11176.0  315.0      8     100.0  False   12   
1   2025-03-15 02:16:16.232  11201.0  315.0      8     100.0  False   12   
2   2025-03-15 02:16:16.632  11309.0  319.0      8     100.0  False   12   
3   2025-03-15 02:16:16.992  11319.0  320.0      8     100.0  False   12   
4   2025-03-15 02:16:17.152  11428.0  320.0      8     100.0  False   12   
..                      ...      ...    ...    ...       ...    ...  ...   
279 2025-03-15 02:17:31.112  10768.0  297.0      7     100.0  False   12   
280 2025-03-15 02:17:31.312  10737.0  301.0      8     100.0  False   12   
281 2025-03-15 02:17:31.672  10877.0  306.0      8     100.0  False   12   
282 2025-03-15 02:17:32.072  10980.0  311.0      8     100.0  False   12   
283 2025-03-15 02:17:32.472  11093.0  312.0      8     100.0  False   12   

    Source                   Time            SessionTime  
0      car 0 days 00:00:00.1

In [42]:
for driver in practice_session.drivers:
    print(driver)

1
4
5
6
7
10
12
14
16
18
22
23
27
30
31
44
55
63
81
87
